In [1]:
import pandas as pd
import numpy as np
import requests
import tabula
import os
import PyPDF2

In [2]:
file = '../raw_data/expl1_spk.pdf'

In [3]:
# unlock file
command = "cp " + file + " temp.pdf; qpdf --password='' --decrypt temp.pdf " + file + "; rm temp.pdf"
os.system(command)

0

# PDFplumber

In [2]:
import pdfplumber
import re



In [17]:
files = ['../raw_data/20210226_Orderabrechnung_-bestaetigung_ISIN_DE000SYM9999_SYMRISE_AG_vom_25_02_2021.pdf',
        '../raw_data/20210205_Orderabrechnung_-bestaetigung_ISIN_DE0007164600_SAP_SE_vom_04_02_2021.PDF',
        '../raw_data/20210205_Orderabrechnung_-bestaetigung_ISIN_CH0012549785_SONOVA_HOLDING_AG_vom_04_02_2021.PDF',
        '../raw_data/20210205_Orderabrechnung_-bestaetigung_ISIN_US0304201033_AMERICAN_WATER_WOR_vom_04_02_2021.PDF',
        '../raw_data/20210220_Orderabrechnung_-bestaetigung_ISIN_DE000EWG2LD7_BOERSE_STUTTGART_S_vom_19_02_2021.PDF']
df = pd.DataFrame()
for file in files:    
    
    with pdfplumber.open(file) as pdf:
        pages = [page.extract_text() for page in pdf.pages]

    match = [p.start() for p in re.finditer('Nominale', pages[0])]
    trans = pages[0][match[0]:].split('\n')
    action = re.findall('Wertpapier Abrechnung (\w+)', pages[0])
    
    if action:
        # if 'Abrechnung' was found
        val = re.findall(r'Stück (\d+)', trans[1])[0]
        wkn = trans[1].replace(')', '').split('(')[1]
        date = re.findall(r'Datum (\d+.\d+.\d+)', pages[0])[0]
        
        # assemble df
        data = [val, wkn, date]
        columns = [action[0], 'WKN', 'Datum']
        
        
        # scan 2nd page
        if action[0] == 'Verkauf':
            buy = [p.start() for p in re.finditer('Ber\wcksichtigte Anschaffungsgesch\wfte', pages[1])][0]
#             columns2 = columns + pages[1][buy:].split('\n')[1].split()
            columns2 = columns + ['Geschäft', 
                                  'Auftragsnr.', 
                                  'Ausführ.-tag', 
                                  'Whr./St.', 
                                  'Nennwert/Stück', 
                                  'AS-Kosten', 
                                  'Erlös', 'ant.Ergebnis',
                                  'Land']
            data2 = data + pages[1][buy:].split('\n')[2].split()
            try:
                new_df = pd.DataFrame([data2], columns=columns2)
            except:
                new_df = (pd.DataFrame([data], columns=columns))
                print(f'couldn"t process 2nd page of file: {file}')
            
        else:
            rate = re.findall(r'Ausf\whrungskurs(\d+,\d+)', pages[0])[0]
            data.append(rate)
            columns.append('Kurs')
            new_df = pd.DataFrame([data], columns=columns)
        
        
        df = pd.concat([df, new_df])


couldn"t process 2nd page of file: ../raw_data/20210220_Orderabrechnung_-bestaetigung_ISIN_DE000EWG2LD7_BOERSE_STUTTGART_S_vom_19_02_2021.PDF


In [18]:
df

,Verkauf,WKN,Datum,Geschäft,Auftragsnr.,Ausführ.-tag,Whr./St.,Nennwert/Stück,AS-Kosten,Erlös,ant.Ergebnis,Land,Kauf,Kurs
0,240,SYM999,25.02.2021,Kauf,2913070700,13.03.2020,Stück,"130,0000","11.021,54-","12.763,25","1.741,71",(D),NaN,NaN
0,NaN,716460,04.02.2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225,"110,54"
0,NaN,A0NJ38,04.02.2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,"137,38"
0,725,EWG2LD,19.02.2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Tabula|

In [8]:
mine = 'https://storage.googleapis.com/bank_price_pdfs/expl1_spk.pdf'
file = '../raw_data/20210226_Orderabrechnung_-bestaetigung_ISIN_DE000SYM9999_SYMRISE_AG_vom_25_02_2021.pdf'
url = 'https://raw.githubusercontent.com/chezou/tabula-py/master/tests/resources/MultiColumn.pdf'
tables = tabula.read_pdf(file, 
                         stream=True,
                         pages='all')

In [9]:
tables

[]

# StringIO

In [18]:
from io import StringIO

# Read each line of the PDF
pdfContent = StringIO(getPDFContent(file).encode("ascii", "ignore"))

NameError: name 'getPDFContent' is not defined

# PyPDF2